Build currents on momentum Grid
==============

In [9]:
import numpy as np
from sftrident import qft



Build grid of kinematic parameters
-----------

In [71]:
def GridShape(Grid):
    #print [np.array(el.shape) for el in Grid]
    return tuple(sum([np.array(el.shape) for el in Grid])-len(Grid)+1)

def BuildGrid(ss,p1x,p1y,p1m,p2x,p2y,p2m):
    SS  = qft.parray(ss)[:,np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis]
    P1x = qft.parray(p1x)[np.newaxis,:,np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis]
    P1y = qft.parray(p1y)[np.newaxis,np.newaxis,:,np.newaxis,np.newaxis,np.newaxis,np.newaxis]
    P1m = qft.parray(p1m)[np.newaxis,np.newaxis,np.newaxis,:,np.newaxis,np.newaxis,np.newaxis]
    P2x = qft.parray(p2x)[np.newaxis,np.newaxis,np.newaxis,np.newaxis,:,np.newaxis,np.newaxis]
    P2y = qft.parray(p2y)[np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis,:,np.newaxis]
    P2m = qft.parray(p2m)[np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis,:]
    return [SS,P1x,P1y,P1m,P2x,P2y,P2m]


ssInit = np.array([3.353])
pa_x = np.linspace(0.0,3.5,10)
pa_y = np.linspace(0.0,3.5,10)
pa_m = np.linspace(0.1,3.5,10)

pb_x = np.linspace(0.0,3.5,10)
pb_y = np.linspace(0.0,3.5,10)
pb_m = np.linspace(-1.0,3.5,10)

kinGridRaw = BuildGrid(ssInit,pa_x,pa_y,pa_m,pb_x,pb_y,pb_m)

kinGrid = kinGridRaw[0]*kinGridRaw[1]*kinGridRaw[2]*kinGridRaw[3]*kinGridRaw[4]*kinGridRaw[5]*kinGridRaw[6]
print kinGrid[0,4,5,6,7,6,5]
print "shape: %s"%(str(GridShape(kinGridRaw)))

228.68797673754005
shape: (1, 10, 10, 10, 10, 10, 10)


Build secondary kinematic parameters (initial particles)
----------

In [44]:
def omegaFromSS(ss):
    #lab
    return (ss**2 - 1.0)/2.0

#initial particles
omega = omegaFromSS(kinGridRaw[0])
print "omega shape: %s"%(str(omega.shape))
print "omega: %s"%omega

E = np.ones(omega.shape)
print "E shape: %s"%(str(E.shape))
print "E: %s"%E

Et = omega + E
pt = omega
print "Et shape: %s"%(str(Et.shape))
print "Et: %s"%Et
print "pt shape: %s"%(str(pt.shape))
print "pt: %s"%pt

#

omega shape: (1, 1, 1, 1, 1, 1, 1)
omega: [[[[[[[5.1213045]]]]]]]
E shape: (1, 1, 1, 1, 1, 1, 1)
E: [[[[[[[1.]]]]]]]
Et shape: (1, 1, 1, 1, 1, 1, 1)
Et: [[[[[[[6.1213045]]]]]]]
pt shape: (1, 1, 1, 1, 1, 1, 1)
pt: [[[[[[[5.1213045]]]]]]]


Build momenta of initial particles
-------------

In [45]:
K=qft.MinkowskiVector([omega,0,0,omega])
P=qft.MinkowskiVector([E,0,0,0])

Build secondary kinematic parameters (final particles)
----------

In [46]:
Pcm = 0.5*(P._0() - P._3()) - kinGridRaw[3] - kinGridRaw[6]
Pcx =  - kinGridRaw[1] - kinGridRaw[4]
Pcy =  - kinGridRaw[2] - kinGridRaw[5]

print "Pcm shape: %s"%(str(Pcm.shape))
print "Pcx shape: %s"%(str(Pcx.shape))
print "Pcy shape: %s"%(str(Pcy.shape))

Pcm shape: (1, 1, 1, 10, 1, 1, 10)
Pcx shape: (1, 10, 1, 1, 10, 1, 1)
Pcy shape: (1, 1, 10, 1, 1, 10, 1)


Build momenta of final particles (under condition)
------------

In [70]:
def physArea(P1m,P2m,P3m):
    return (P1m>0) & (P2m>0) & (P3m>0) 


def BuildMomentum(px,py,pm):
    pp = (px**2 + py**2 + 1)/(4.0*pm)
    return qft.MinkowskiVector([pp+pm,px,py,pp-pm])


def buildAllMom(p1x,p1y,p1m,p2x,p2y,p2m,p3x,p3y,p3m):
    p1mT = p1m[p1m>0 & p2m>0 & p3m>0]
    p2mT = p2m[p1m>0 & p2m>0 & p3m>0]
    p2mT = p2m[p1m>0 & p2m>0 & p3m>0]
    P1 = BuildMomentum(p1x,p1y,p1m)
    P2 = BuildMomentum(p2x,p2y,p2m)
    P3 = BuildMomentum(p3x,p3y,p3m)
    return P1,P2,P3


physAr = physArea(kinGridRaw[3],kinGridRaw[6],Pcm)
print physAr.shape

Pa,Pb,Pc = buildAllMom(kinGridRaw[1],kinGridRaw[2],kinGridRaw[3],kinGridRaw[4],kinGridRaw[5],kinGridRaw[6],Pcx,Pcy,Pcm)
print Pa*Pb
#Pa = BuildMomentum(kinGridRaw[1],kinGridRaw[2],kinGridRaw[3])
#Pb = BuildMomentum(kinGridRaw[4],kinGridRaw[5],kinGridRaw[6])
#Pc = BuildMomentum(Pcx,Pcy,Pcm)

(1, 1, 1, 10, 1, 1, 10)


TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [66]:
def allowedMass(kinPara):
    mom = BuildMomentum(*kinPara)
    res = mom*mom
    #print "type: %s"%(res.shape)
    #print "shape: %s"%(str(res.shape))
    return res


def Mass(px,py,pm):
    """
    calcs the mass on Grid where pm>0 (else mass=0)
    """
    pmAllowed = pm[pm>0]
    res = allowedMass([px,py,pmAllowed])
    return res
    
m2= Mass(kinGridRaw[4],kinGridRaw[5],kinGridRaw[6])
#allowedMass([kinGridRaw[1],kinGridRaw[2],kinGridRaw[3]])
m1 = Mass(kinGridRaw[1],kinGridRaw[2],kinGridRaw[3])
print np.asarray(m1).shape
print np.asarray(m2).shape
#Pa = BuildMomentum(kinGridRaw[1],kinGridRaw[2],kinGridRaw[3])
#Pb = BuildMomentum(kinGridRaw[4],kinGridRaw[5],kinGridRaw[6])
#Pc = BuildMomentum(Pcx,Pcy,Pcm)


(1, 10, 10, 1, 1, 1, 10)
(1, 1, 1, 1, 10, 10, 7)


(1, 10, 1, 10, 10, 1, 10)
(1, 10, 1, 1, 10, 1, 1)
